In [1]:
from gensim.utils import tokenize
from langdetect import detect
import string
from itertools import islice
import numpy as np
from scipy.sparse import csr_matrix
import smart_open
import faiss

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.linalg import Vectors, VectorUDT

import pickle

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1566899959131_0008,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model/svd"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"
data_path = "s3://onai-ml-dev-eu-west-1/company2vec/data"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/bow/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
with smart_open.open(f"{model_path}/V/mat.np", "rb") as f:
    V = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
len(word2id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

42829

In [6]:
with smart_open.open(f"{model_path}/X/X_pca.np", "rb") as f:
    X_pca = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
with smart_open.open(f"{model_path}/X/company_info.np", "rb") as f:
    description = np.load(f, allow_pickle=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
n,d = X_pca.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
index = faiss.IndexFlatL2(d)
index.add(X_pca)
print(index.ntotal)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3381364

In [10]:
company2id = {}
for i,desc in enumerate(description):
    company2id[desc[0]] = i

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def query_peers(query_company, result_size=100):
    query_vector = X_pca[np.newaxis, company2id[query_company]]
    
    desc_info = description[company2id[query_company]]
    industry = desc_info[-2]
    country = desc_info[-3]
    
    D, I = index.search(query_vector, result_size)
    
    ret = []
    added = set()

    for i,el in enumerate(I[0]):
        if description[el,-2] == industry and description[el,-3] == country:
            el = (D[0, i], description[el])
            if repr(el) not in added:
                ret.append(el)
                added.add(repr(el))
        if len(ret) >= result_size:
            break

    # TODO: once we have country embeddings, order by them
    if len(ret) < result_size:
        for i,el in enumerate(I[0]):
            if description[el,-2] == industry:
                el = (D[0, i], description[el])
                if repr(el) not in added:
                    ret.append(el)
                    added.add(repr(el))
            if len(ret) >= result_size:
                break

    # TODO: once we have industry class embeddings, order by them
    if len(ret) < result_size:
        for i,el in enumerate(I[0]):
            if description[el,-3] == country:
                el = (D[0, i], description[el])
                if repr(el) not in added:
                    ret.append(el)
                    added.add(repr(el))
            if len(ret) >= result_size:
                break

    if len(ret) < result_size:
        for i,el in enumerate(I[0]):
            el = (D[0, i], description[el])
            if repr(el) not in added:
                ret.append(el)
                added.add(repr(el))
            if len(ret) >= result_size:
                break
                
    return ret


def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == company:
            return i+1

    return -1


def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
for el in query_peers(106335):
    print(el)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.0, array([106335, 'Ford Motor Company',
       'Ford Motor Company designs, manufactures, markets, and services a range of Ford cars, trucks, sport utility vehicles, and electrified vehicles worldwide. It operates through three segments: Automotive, Mobility, and Ford Credit. The Automotive segment sells Ford and Lincoln vehicles, service parts, and accessories through distributors and dealers, as well as through dealerships to commercial fleet customers, daily rental car companies, and governments. The Mobility segment designs and builds mobility services; and provides self-driving systems development and vehicle integration, autonomous vehicle research and engineering, and autonomous vehicle transportation-as-a-service network development services. The Ford Credit segment primarily engages in vehicle-related financing and leasing activities to and through automotive dealers. It provides retail installment sale contracts for new and used vehicles; and direct financing leases for ne

In [12]:
public_labels = [[2444485, 9429145, 93196, 9870522, 920391],
[645782, 11809880, 392154],
[875173, 237655379, 931146],
[813607, 100231, 357076],
[380011, 357076, 324490],
[882889, 127202, 5478606, 1025390, 645782],
[46329052, 189096, 915379, 46895276, 877008],
[106335, 319676, 377732, 61206100],
[877769, 874042, 780678, 953488, 883809, 875295],
[34049884, 882155, 30428758, 315394],
[254287477, 883300, 30614595],
[5600108, 285880, 5433540, 878697, 35650],
[876031, 410182, 874470, 874191, 879732, 5395336]]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
for public_label in public_labels:
    query_id = public_label[0]
    results = [el[1][0] for el in query_peers(query_id)]
    ranks = []
    
    for peer in public_label[1:]:
        ranks.append(find_peer_in_result(results, peer))
        ranks_default.append(find_peer_in_result(results, peer))

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    #
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 81, -1, -1, -1]

[645782, -1, -1]

[875173, 93, -1]

[813607, -1, 5]

[380011, -1, 38]

[882889, -1, -1, -1, 40]

[46329052, -1, -1, -1, -1]

[106335, -1, -1, 2]

[877769, 18, 80, -1, -1, 52]

[34049884, 6, -1, -1]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1]

[876031, -1, -1, -1, 53, -1]

[3, 1, 2, 5, 0, 31]
0.23055555555555554

In [20]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
for public_label in public_labels:
    query_id = public_label[0]
    results = [el[1][0] for el in query_peers(query_id, 500)]
    ranks = []
    
    for peer in public_label[1:]:
        ranks.append(find_peer_in_result(results, peer))
        ranks_default.append(find_peer_in_result(results, peer))

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    #
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 82, -1, 410, -1]

[645782, 403, -1]

[875173, 121, 406]

[813607, -1, 10]

[380011, 443, 58]

[882889, 4, -1, -1, 48]

[46329052, -1, -1, -1, -1]

[106335, 291, -1, 2]

[877769, 18, 103, 25, -1, 75]

[34049884, 6, -1, -1]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1]

[876031, -1, 405, 117, 63, 322]

[4, 1, 2, 4, 10, 21]
0.21444444444444444